In [377]:
import pandas as pd
from nltk.corpus import wordnet 
import wordsegment
from wordsegment import segment
import spacy
import re

In [382]:
nlp = spacy.load('en')

In [2]:
url = '/Users/FQ/Downloads/finalDataset.csv'
df1 = pd.read_csv(url)

In [3]:
syns = wordnet.synsets("program")

In [4]:
w1 = wordnet.synset('run.v.01') # v here denotes the tag verb 
w2 = wordnet.synset('sprint.v.01') 
print(w1.wup_similarity(w2)) 

0.8571428571428571


In [5]:
#This cell returns error since hashtags can not be recognized by wordnet as they are not words. 
w1 = wordnet.synset(df1["Hashtag"][1]+".n.01")
w2 = wordnet.synset(df1["Hashtag"][2]+".n.01")
print(w1.wup_similarity(w2)) 

WordNetError: no lemma 'fitnessgoals' with part of speech 'n'

In [77]:
#importing all categories into memory
cats = list(set(df1["Category"]))

In [375]:
#removing &, /, , 
for i in range(len(cats)):
    cats[i] = re.sub(r"\/", " ", cats[i])
    cats[i] = re.sub("\&", "", cats[i])
    cats[i] = re.sub("\,", "", cats[i])

cats = [cat.lower() for cat in cats]    

In [379]:
spacy download en_vectors_web_lg

SyntaxError: invalid syntax (<ipython-input-379-e79319a2e687>, line 1)

In [381]:
nlp = spacy.load('en_core_web_lg')
#nlp = spacy.load('en_vectors_web_lg')

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [383]:
#Vectorization is different for upper and lower case
w1 = nlp("TV Media Entertainment")
w2 = nlp("TV")
w3 = nlp("Media")
w4 = nlp("Entertainment")
vec1 = w1.vector
vec2 = w2.vector
vec3 = (w2.vector+w3.vector+w4.vector)
print("distance between \"TV\" and \"TV Media Entertainment\":",cosine_similarity(vec2.reshape(1,len(vec2)),vec1.reshape(1,len(vec1))))
print("distance between \"TV\" and \"TV\"+ \"Media\"+ \"Entertainment\":",cosine_similarity(vec2.reshape(1,len(vec2)),vec3.reshape(1,len(vec3))))
w1 = nlp("TV Media Entertainment".lower())
w2 = nlp("TV".lower())
w3 = nlp("Media".lower())
w4 = nlp("Entertainment".lower())
vec1 = w1.vector
vec2 = w2.vector
vec3 = (w2.vector+w3.vector+w4.vector)
print("distance between \"tv\" and \"tv media entertainment\":",cosine_similarity(vec2.reshape(1,len(vec2)),vec1.reshape(1,len(vec1))))
print("distance between \"tv\" and \"tv\"+ \"media\"+ \"entertainment\":",cosine_similarity(vec2.reshape(1,len(vec2)),vec3.reshape(1,len(vec3))))

distance between "TV" and "TV Media Entertainment": [[0.6172439]]
distance between "TV" and "TV"+ "Media"+ "Entertainment": [[0.8729253]]
distance between "tv" and "tv media entertainment": [[0.62123775]]
distance between "tv" and "tv"+ "media"+ "entertainment": [[0.8558471]]


In [79]:
#the lesson is in the case of compound word it is better to consider the sum of vectors
split_cats = [cat.split(" ") for cat in cats]

In [107]:
def f(x):
    vec = np.zeros(384)
    for word in x:
        if word!= "": 
            tok_word = nlp(word)
            vec+=tok_word.vector  
    return vec

In [109]:
split_cats_vec = list(map(lambda x: f(x), split_cats))

In [69]:
from nltk.cluster import KMeansClusterer
import nltk

In [130]:
NUM_CLUSTERS = 30
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25, avoid_empty_clusters= True)
assigned_clusters = kclusterer.cluster(split_cats_vec, assign_clusters=True)
print(assigned_clusters)

[10, 29, 11, 21, 29, 24, 1, 14, 8, 14, 21, 27, 23, 20, 22, 21, 22, 3, 5, 25, 19, 21, 24, 6, 29, 17, 23, 14, 28, 24, 11, 18, 22, 22, 16, 19, 13, 6, 13, 22, 4, 24, 6, 24, 5, 25, 8, 10, 5, 25, 6, 22, 23, 20, 17, 9, 4, 9, 0, 22, 15, 19, 26, 12, 22, 7, 2, 22, 22, 8]


In [131]:
cls = [[] for i in range(NUM_CLUSTERS)]
for i, cls_num in enumerate(assigned_clusters):  
    cls[cls_num].append(cats[i])

In [132]:
cls

[['love'],
 ['interieur'],
 ['general'],
 ['jewelry'],
 ['craft', 'realstate'],
 ['inspirational', 'religious', 'personal'],
 ['legal law', 'small business', 'feng shui', 'financial services'],
 ['podcast'],
 ['photography  videography', 'photography videography', 'home decor'],
 ['wedding', 'life coaching'],
 ['political', 'medical'],
 ['film  cinema', 'film cinema'],
 ['art'],
 ['meme', 'ecommerce'],
 ['nature', 'furniture', 'nutrition'],
 ['finance'],
 ['clothing'],
 ['poetry', 'psychology'],
 ['influencer'],
 ['realestate', 'luxury', 'travel'],
 ['beauty', 'tv media entertainment'],
 ['music', 'fitness', 'agency', 'aviation'],
 ['tattoos',
  'kids',
  'sports',
  'pets',
  'weddings',
  'apps',
  'events',
  'electronics',
  'cars',
  'animals'],
 ['modelling', 'writing', 'painting'],
 ['technology', 'wine', 'fashion', 'barber', 'food'],
 ['fishing', 'vaping', 'fundraising'],
 ['marketing'],
 ['advertising'],
 ['yoga'],
 ['smartphone  gadgets', 'food resturants', 'food  resturants'

In [315]:
dict_cat={}
for i in range(69):
    for j in range(i+1,70):
        u = split_cats_vec[i]
        v = split_cats_vec[j]
        dist = cosine_similarity(u.reshape(1,384),v.reshape(1,384))
        if dist >= 0.8:
            dict_cat[(i,j)] = 1
        else:
            dict_cat[(i,j)] = 0

In [317]:
l = np.where(np.array(list(dict_cat.values()))==1)
keys = list(dict_cat.keys())
for i in l[0]:
    print(cats[keys[i][0]], " ",cats[keys[i][1]]) 

political   inspirational
political   religious
political   medical
political   personal
film  cinema   photography  videography
film  cinema   film cinema
film  cinema   photography videography
film  cinema   home decor
music   photography  videography
music   aviation
music   wine
music   nutrition
music   luxury
music   photography videography
music   tv media entertainment
food resturants   food  resturants
technology   fitness
technology   jewelry
technology   wine
technology   psychology
nature   photography  videography
nature   furniture
nature   beauty
nature   aviation
nature   nutrition
nature   photography videography
nature   tv media entertainment
nature   life coaching
nature   art
photography  videography   film cinema
photography  videography   photography videography
photography  videography   art
photography  videography   home decor
furniture   ecommerce
fitness   aviation
fitness   tv media entertainment
advertising   writing
advertising   wedding
advertising   lif

In [137]:
from wordsegment import load, segment
load()

In [133]:
tags = df1["Hashtag"]
len(tags)

63434

In [139]:
split_tags = list(map(lambda x: segment(x), tags))

In [141]:
split_tags_vec = list(map(lambda x: f(x), split_tags))

In [142]:
#np.save('split_tags_vec.npy', split_tags_vec)
#tst = np.load('split_tags_vec.npy')

In [289]:
NUM_CLUSTERS = 30
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25, avoid_empty_clusters= True)
assigned_clusters = kclusterer.cluster(split_cats_vec, assign_clusters=True)

In [293]:
len(split_tags_vec)

63434

In [294]:
dict_tag={}
for i in range(63433):
    for j in range(i+1,63434):
        u = split_tags_vec[i]
        v = split_tags_vec[j]
        dist = cosine_similarity(u.reshape(1,384),v.reshape(1,384))
        if dist >= 0.8:
            dict_tag[(i,j)] = 1
        else:
            dict_tag[(i,j)] = 0
            

KeyboardInterrupt: 

In [306]:
u = split_tags_vec[38]
v = split_tags_vec[20712]
cosine_similarity(u.reshape(1,384),v.reshape(1,384))

array([[0.74585302]])

In [309]:
split_tags[38]
split_tags[20712]

['cristo']

In [303]:
keys = list(dict_tag.keys())
list(dict_tag.values())

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [254]:
l = np.where(np.array(list(dict_tag.values()))==1)
keys = list(dic.keys())
for i in l[0]:
    print(cats[keys[i][0]], " ",cats[keys[i][1]]) 

political   inspirational
political   religious
political   medical
political   personal
film  cinema   photography  videography
film  cinema   film cinema
film  cinema   photography videography
film  cinema   home decor
music   photography  videography
music   aviation
music   wine
music   nutrition
music   luxury
music   photography videography
music   tv media entertainment
food resturants   food  resturants
technology   fitness
technology   jewelry
technology   wine
technology   psychology
nature   photography  videography
nature   furniture
nature   beauty
nature   aviation
nature   nutrition
nature   photography videography
nature   tv media entertainment
nature   life coaching
nature   art
photography  videography   film cinema
photography  videography   photography videography
photography  videography   art
photography  videography   home decor
furniture   ecommerce
fitness   aviation
fitness   tv media entertainment
advertising   writing
advertising   wedding
advertising   lif

In [177]:
u=split_tags_vec[1]
v=split_tags_vec[2]
cosine_similarity(v.reshape(1,384),u.reshape(1,384))

array([[0.84389376]])

In [155]:
cls = [[] for i in range(NUM_CLUSTERS)]
for i, cls_num in enumerate(assigned_clusters):  
    cls[cls_num].append(df1["Category"][i])

In [325]:
#import collections
d = df1.groupby("Category")["Hashtag"].apply(lambda x: list(x))

array([list(['lovehorses', 'hairart', 'hairpost', 'boating', 'boatlife', 'goprooftheday', 'gopro4', 'freshcuties', 'yorubacuties', 'remyo', 'remyondeck', 'remyontherocks', 'remyornothing', 'remyothman', 'remyowen', 'remyontherecord', 'remyoasis', 'remyoliver', 'remyoakland', 'remyolivier', 'remyofinstagram', 'remyornot', 'remyoliverbleu', 'remyog', 'remyonracine', 'remyology', 'remyonice', 'remyorganic', 'remyontherun', 'remyotto', 'remyofnyc', 'remyongstrikesback', 'remyonme', 'remyourdog', 'remyong', 'remyout', 'remyocnl', 'remyonline', 'remyonchill', 'remyornah', 'remyonack', 'remyonsnap', 'remyortiz', 'remyontheroad', 'remyondarocks', 'remyonrocks', 'remyorpatron', 'remyofficial', 'remyofcourse', 'instaslayyers', 'instaslaynaija', 'instaslayerz', 'instaslaya', 'instaslaying', 'instaslayed', 'instaslayqueen', 'instaslayer😍', 'instaslayage', 'instaslayer🔥', 'instaslayy', 'instaslayt', 'instaslays', 'instaslayersworld', 'instaslayin', 'instaslayqueens', 'instaslayersjoin', 'instaslays

In [352]:
df_cat2tag = pd.DataFrame()

In [354]:
df_cat2tag["Category"] = ['Advertising', 'Agency', 'Animals', 'Apps', 'Art', 'Aviation', 'Barber',
       'Beauty', 'Cars', 'Clothing', 'Craft', 'Ecommerce', 'Electronics',
       'Events', 'Fashion', 'Feng shui', 'Film/ Cinema', 'Film/Cinema',
       'Finance', 'Financial Services', 'Fishing', 'Fitness', 'Food',
       'Food/ Resturants', 'Food/Resturants', 'FundRaising', 'Furniture',
       'General', 'Home Decor', 'Influencer', 'Inspirational', 'Kids', 'LOVE',
       'Legal/Law', 'Life Coaching', 'Luxury', 'Marketing', 'Medical',
       'Modelling', 'Music', 'Nature', 'Nutrition', 'Painting', 'Personal',
       'Pets', 'Photography/ Videography', 'Photography/Videography',
       'PodCast', 'Poetry', 'Political', 'Psychology', 'RealEstate',
       'Realstate', 'Religious', 'Small Business', 'Smartphone & Gadgets',
       'Sports', 'TV, Media, Entertainment', 'Tattoos', 'Technology', 'Travel',
       'Vaping', 'Wedding', 'Wine', 'Writing', 'Yoga', 'interieur', 'jewelry',
       'meme', 'weddings']

In [360]:
df_cat2tag["tags"] = list(d.values)

In [363]:
df_cat2tag["n_tags"] = [len(x) for x in list(d.values)]

In [364]:
df_cat2tag

,Category,tags,n_tags
0,Advertising,"[lovehorses, hairart, hairpost, boating, boatl...",971
1,Agency,"[norwich, diss, eastanglia, northnorfolk, jobs...",337
2,Animals,"[nationalpuppyday, worldofcutepets, pups, pups...",665
3,Apps,"[newrestaurant, muscat, california, california...",396
4,Art,"[art, artwork, painting, gallery, artcurator, ...",3086
5,Aviation,"[neverstoplearning, neverquit, nevertooold, la...",28
6,Barber,"[hair, hairstyle, haircut, hairstylist, hairco...",245
7,Beauty,"[cosmetics, cosmetic, benefitcosmetics, cosmet...",3115
8,Cars,"[car, cars, carporn, carnaval, dogfood, 9inlif...",1072
9,Clothing,"[streetwear, streetwearkl, streetwearmalaysia,...",1273


In [4]:
!pip install bert-serving-server

In [5]:
!pip install bert-serving-client 

In [27]:
f = open("/Users/FQ/DownloadS/uncased_L-12_H-768_A-12/bert_config.json", "r")
print(f.read)

<built-in method read of _io.TextIOWrapper object at 0x630e64a68>


In [35]:
!bert-serving-start -model_dir="/Users/FQ/Downloads/uncased_L-12_H-768_A-12/" -num_worker=4 
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

usage: /Users/FQ/anaconda3/bin/bert-serving-start -model_dir=/Users/FQ/Downloads/uncased_L-12_H-768_A-12/ -num_worker=4
                 ARG   VALUE
__________________________________________________
           ckpt_name = bert_model.ckpt
         config_name = bert_config.json
                cors = *
                 cpu = False
          device_map = []
       do_lower_case = True
  fixed_embed_length = False
                fp16 = False
 gpu_memory_fraction = 0.5
       graph_tmp_dir = None
    http_max_connect = 10
           http_port = None
        mask_cls_sep = False
      max_batch_size = 256
         max_seq_len = 25
           model_dir = /Users/FQ/Downloads/uncased_L-12_H-768_A-12/
          num_worker = 4
       pooling_layer = [-2]
    pooling_strategy = REDUCE_MEAN
                port = 5555
            port_out = 5556
       prefetch_size = 10
 priority_batch_size = 16
show_tokens_to_client = False
     tuned_model_dir = None
             verbose = False
             

I:SINK:[__i:_ru:348]:send config	client b'ccac2d44-587f-4469-97bb-b33d419fff3d'
I0801 02:53:17.048578 123145356152832 __init__.py:348] send config	client b'ccac2d44-587f-4469-97bb-b33d419fff3d'
^C
I:VENTILATOR:[__i:clo: 89]:shutting down...
I0801 02:53:47.741413 4607133120 __init__.py:89] shutting down...
Process BertSink-2:
I:SINK:[__i:clo:274]:shutting down...
I0801 02:53:47.748827 123145356152832 __init__.py:274] shutting down...
I:SINK:[__i:clo:279]:terminated!
I0801 02:53:47.751540 123145356152832 __init__.py:279] terminated!
I:WORKER-0:[__i:clo:480]:shutting down...
I0801 02:53:47.751696 123145356152832 __init__.py:480] shutting down...
I:WORKER-0:[__i:clo:485]:terminated!
I0801 02:53:47.833030 123145356152832 __init__.py:485] terminated!
I:WORKER-1:[__i:clo:480]:shutting down...
I0801 02:53:47.833208 123145356152832 __init__.py:480] shutting down...
I:WORKER-1:[__i:clo:485]:terminated!
I0801 02:53:47.909291 123145356152832 __init__.py:485] terminated!
I:WORKER-2:[__i:clo:480]:sh

KeyboardInterrupt: 

In [93]:
from bert_serving.client import BertClient
bc = BertClient()
sent_encod = bc.encode(['I feel terrible', 'It is awesome day', 'I am sick'])

In [96]:
cosine_similarity(sent_encod[0].reshape(1,-1),sent_encod[1].reshape(1,-1))

array([[0.74521]], dtype=float32)

In [70]:
import pandas as pd
import re

In [56]:
url = '/Users/FQ/Downloads/finalDataset.csv'
df1 = pd.read_csv(url)

In [59]:
cats = list(set(df1["Category"]))

In [71]:
for i in range(len(cats)):
    cats[i] = re.sub(r"\/", " ", cats[i])
    cats[i] = re.sub("\&", "", cats[i])
    cats[i] = re.sub("\,", "", cats[i])

cats = [cat.lower() for cat in cats]    

In [73]:
cats_encod = bc.encode(cats)

In [62]:
from nltk.cluster import KMeansClusterer
import nltk

In [80]:
NUM_CLUSTERS = 35
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25, avoid_empty_clusters= True)
assigned_clusters = kclusterer.cluster(cats_encod, assign_clusters=True)

In [81]:
cls = [[] for i in range(NUM_CLUSTERS)]
for i, cls_num in enumerate(assigned_clusters):  
    cls[cls_num].append(cats[i])

In [82]:
cls

[['food  resturants', 'food resturants'],
 ['vaping'],
 ['interieur', 'realestate'],
 ['film  cinema', 'film cinema'],
 ['tv media entertainment'],
 ['meme'],
 ['ecommerce', 'feng shui'],
 ['photography  videography', 'photography videography'],
 ['influencer', 'life coaching'],
 ['realstate'],
 ['sports', 'events'],
 ['financial services'],
 ['small business'],
 ['finance', 'legal law'],
 ['home decor'],
 ['music', 'craft', 'wine'],
 ['barber'],
 ['fishing', 'aviation', 'electronics', 'political'],
 ['cars', 'animals'],
 ['agency'],
 ['medical', 'art', 'fashion', 'inspirational'],
 ['apps', 'pets', 'personal', 'podcast'],
 ['jewelry', 'tattoos', 'smartphone  gadgets'],
 ['food', 'kids'],
 ['religious'],
 ['writing',
  'painting',
  'marketing',
  'technology',
  'advertising',
  'fundraising',
  'modelling'],
 ['nature'],
 ['general'],
 ['furniture', 'clothing'],
 ['beauty', 'luxury'],
 ['wedding', 'weddings'],
 ['yoga', 'fitness', 'nutrition', 'psychology'],
 ['love'],
 ['poetry'],
 

In [99]:
bc.close()